In [17]:
import numpy as np
import random as rd
import scipy.stats as sc

l = 150
n = 1000000
B = 41.5
x_exit = 0.5 * B
phi = 37
d70m = 2.08e-4
dn = 4.5
v = 1.33e-6
g = 9.81
yw = 10
ys = 26.50

# Water level
data = np.loadtxt('Cumulative_density_function_water_level.txt', delimiter=' ', unpack=True)

def lognorm_params(mode, stddev):
    p = np.poly1d([1, -1, 0, 0, -(stddev/mode)**2])
    r = p.roots
    sol = r[(r.imag == 0) & (r.real > 0)].real
    shape = np.sqrt(np.log(sol))
    scale = mode * sol
    return shape, scale

Pfu = []
Pfh = []
Pfp = []
for i in range(l):
    # Generate Stochastic Lognormal
    sigma, scale = lognorm_params(25, 0.5)
    D = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(4.5, 0.5)
    d = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(7.52e-4, 0.5 * 7.52e-4)
    k = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(1.00e-6, 0.5 * 1.00e-6)
    kh = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(20, 2)
    Lf = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(0.5, 0.1)
    ich = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(2.8e-4, 0.12*2.8e-4)
    d70 = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    # Generate Stochastic Normal
    hp = sc.norm.rvs(3.50, 0.1, n)
    ysat = (2.5 * 14.5 + 2 * 15) / (dn)
    ysat = sc.norm.rvs(ysat, 0.05 * ysat, n)
    mu = sc.norm.rvs(1.00, 0.1, n)
    mp = sc.norm.rvs(1.0, 0.12, n)
    
    # Generate water level
    h = []
    for q in range(n):
        w = rd.random()
        h.append(np.interp(w, data[1], data[0]))
    
    # Uplift
    lambda_h = ((k * D * d) / kh) ** 0.5
    lambda_ = (lambda_h / (Lf + B + lambda_h)) * np.exp((0.5 * B - x_exit)/lambda_h)
    phi_exit = hp + lambda_ * (h - hp)
    dphi = phi_exit - hp
    dphi_c = d * ((ysat - yw) / yw)
    Zu = mu * dphi_c - dphi
    
    # Heave
    ic = (phi_exit - hp) / d
    Zh = ich - ic
    
    # Piping
    L = Lf + 0.5 * B + x_exit + 1
    F1 = 0.25 * ((ys / yw) - 1) * np.tan(np.radians(phi))
    F2 = (d70m / (((v * k * L) / g) ** (1/3))) * (d70 / d70m) ** 0.4
    F3 = 0.91 * (D / L) ** ((0.28 / ((D / L)**2.8 - 1)) + 0.04)
    H = h - hp - 0.3 * d
    Hcp = F1 * F2 * F3 * L
    Zp = mp * Hcp - H
    
    # Analysis
    failu = 0
    failh = 0
    failp = 0
    for q in range(n):
        if(Zu[q] < 0):
            failu += 1
        if(Zh[q] < 0):
            failh += 1
        if(Zp[q] < 0):
            failp += 1
    
    if(failu > 0):
        Pfu.append(failu / n)
    else:
        Pfu.append(0)
        
    if(failh > 0):
        Pfh.append(failh / n)
    else:
        Pfh.append(0)
        
    if(failp > 0):
        Pfp.append(failp / n)
    else:
        Pfp.append(0)
        
    print(i, "Pfu:", Pfu[i], "Pfh:", Pfh[i], "Pfp:", Pfp[i], "Pf:", np.min([Pfu[i],Pfh[i],Pfp[i]]), "FoS:", (1.3e-5)/(np.min([Pfu[i],Pfh[i],Pfp[i]])))
print("TOTAL Pfu:", np.average(Pfu), "Pfh:", np.average(Pfh), "Pfp:", np.average(Pfp), "UPPER Pf:", np.min([np.average(Pfu),np.average(Pfh),np.average(Pfp)]))

0 Pfu: 0.022966 Pfh: 0.013104 Pfp: 7e-06 Pf: 7e-06 FoS: 1.857142857142857
1 Pfu: 0.023066 Pfh: 0.01315 Pfp: 1.4e-05 Pf: 1.4e-05 FoS: 0.9285714285714285
2 Pfu: 0.022937 Pfh: 0.013342 Pfp: 1.2e-05 Pf: 1.2e-05 FoS: 1.0833333333333333
3 Pfu: 0.02284 Pfh: 0.013108 Pfp: 1.3e-05 Pf: 1.3e-05 FoS: 1.0
4 Pfu: 0.022953 Pfh: 0.013029 Pfp: 7e-06 Pf: 7e-06 FoS: 1.857142857142857
5 Pfu: 0.022863 Pfh: 0.012798 Pfp: 7e-06 Pf: 7e-06 FoS: 1.857142857142857
6 Pfu: 0.023196 Pfh: 0.013074 Pfp: 1.2e-05 Pf: 1.2e-05 FoS: 1.0833333333333333
7 Pfu: 0.023236 Pfh: 0.012977 Pfp: 6e-06 Pf: 6e-06 FoS: 2.1666666666666665
8 Pfu: 0.023164 Pfh: 0.013165 Pfp: 9e-06 Pf: 9e-06 FoS: 1.4444444444444444
9 Pfu: 0.023225 Pfh: 0.013209 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
10 Pfu: 0.023008 Pfh: 0.013113 Pfp: 1.2e-05 Pf: 1.2e-05 FoS: 1.0833333333333333
11 Pfu: 0.02298 Pfh: 0.013 Pfp: 1.2e-05 Pf: 1.2e-05 FoS: 1.0833333333333333
12 Pfu: 0.023265 Pfh: 0.013286 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
13 Pfu: 0.023004 Pfh: 

110 Pfu: 0.023128 Pfh: 0.01313 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
111 Pfu: 0.023119 Pfh: 0.013306 Pfp: 8e-06 Pf: 8e-06 FoS: 1.625
112 Pfu: 0.023051 Pfh: 0.013234 Pfp: 8e-06 Pf: 8e-06 FoS: 1.625
113 Pfu: 0.022963 Pfh: 0.013017 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
114 Pfu: 0.023098 Pfh: 0.01316 Pfp: 1.1e-05 Pf: 1.1e-05 FoS: 1.1818181818181819
115 Pfu: 0.023002 Pfh: 0.012936 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
116 Pfu: 0.023106 Pfh: 0.013256 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
117 Pfu: 0.023038 Pfh: 0.012971 Pfp: 1.4e-05 Pf: 1.4e-05 FoS: 0.9285714285714285
118 Pfu: 0.022967 Pfh: 0.013119 Pfp: 1.2e-05 Pf: 1.2e-05 FoS: 1.0833333333333333
119 Pfu: 0.022982 Pfh: 0.013124 Pfp: 1.5e-05 Pf: 1.5e-05 FoS: 0.8666666666666666
120 Pfu: 0.023234 Pfh: 0.013152 Pfp: 1.2e-05 Pf: 1.2e-05 FoS: 1.0833333333333333
121 Pfu: 0.023185 Pfh: 0.013194 Pfp: 7e-06 Pf: 7e-06 FoS: 1.857142857142857
122 Pfu: 0.022972 Pfh: 0.013054 Pfp: 1e-05 Pf: 1e-05 FoS: 1.2999999999999998
123 Pfu:

In [18]:
Pfp

[7e-06,
 1.4e-05,
 1.2e-05,
 1.3e-05,
 7e-06,
 7e-06,
 1.2e-05,
 6e-06,
 9e-06,
 1e-05,
 1.2e-05,
 1.2e-05,
 1e-05,
 1.5e-05,
 1.2e-05,
 6e-06,
 1.4e-05,
 1.5e-05,
 1.1e-05,
 1e-05,
 1.2e-05,
 1.3e-05,
 1.2e-05,
 8e-06,
 1e-05,
 5e-06,
 1.3e-05,
 6e-06,
 8e-06,
 6e-06,
 9e-06,
 1.2e-05,
 1.1e-05,
 1e-05,
 1.3e-05,
 1e-05,
 1.3e-05,
 1.2e-05,
 9e-06,
 1.8e-05,
 1.3e-05,
 1e-05,
 7e-06,
 1.7e-05,
 1.2e-05,
 1.4e-05,
 1.3e-05,
 6e-06,
 9e-06,
 9e-06,
 9e-06,
 8e-06,
 8e-06,
 9e-06,
 1.3e-05,
 1.1e-05,
 1.1e-05,
 1.5e-05,
 1.1e-05,
 3e-06,
 8e-06,
 1.1e-05,
 6e-06,
 1.6e-05,
 1.2e-05,
 4e-06,
 1.2e-05,
 7e-06,
 1.5e-05,
 1.4e-05,
 1e-05,
 1e-05,
 1.1e-05,
 8e-06,
 7e-06,
 7e-06,
 7e-06,
 1.5e-05,
 1.6e-05,
 1.2e-05,
 9e-06,
 1.2e-05,
 9e-06,
 1.2e-05,
 1.3e-05,
 7e-06,
 1.4e-05,
 1.2e-05,
 1.5e-05,
 9e-06,
 1e-05,
 1e-05,
 5e-06,
 6e-06,
 1e-05,
 1.4e-05,
 7e-06,
 1e-05,
 8e-06,
 6e-06,
 1.6e-05,
 1.5e-05,
 1.2e-05,
 1.3e-05,
 1.8e-05,
 9e-06,
 1.3e-05,
 1.5e-05,
 1e-05,
 6e-06,
 1e-05,
 8